In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn import ensemble

from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import matthews_corrcoef

from sklearn.svm import SVC

In [2]:
data = pd.read_csv('tatarstan_message_2.csv', encoding='utf8', sep=',')
print(data.columns)
data = data[['categories', 'description_stem']]
data.head(3)

Index(['level_0', 'index', 'categories', 'description', 'worker',
       'description_prep', 'num_words', 'description_stem'],
      dtype='object')


,categories,description_stem
0,Благоустройство территории,трасса нижнекамск чистополь лесополоса пгт кам...
1,Благоустройство территории,двор дом ленинградский отсутствовать парковочн...
2,Содержание и ремонт муниципальных дорог,добрый день осуществляться уборка снег центр г...


LogisticRegression

In [3]:
#определяем модель "мешка слов"
data = data.dropna()
text_train = data['description_stem'].to_list()

vect = CountVectorizer(min_df=5).fit(text_train) #игнорировать слова, которые встречаются менее чем в min_df документах
#CountVectorizer по умолчанию сам производит токенизацию и выкидывает слова с длиной меньшей чем два.
print(vect.vocabulary_)

X_train = vect.transform(text_train)
y_train = data['categories'].to_list()

{'трасса': 4785, 'нижнекамск': 2490, 'чистополь': 5214, 'лесополоса': 1943, 'пгт': 2976, 'камский': 1589, 'поляна': 3320, 'проезжий': 3656, 'частью': 5183, 'опасный': 2729, 'нависать': 2251, 'накрениться': 2293, 'дерево': 951, 'снегопад': 4305, 'мочь': 2196, 'выдержать': 647, 'рухнуть': 4031, 'проходить': 3740, 'автотранспорт': 48, 'двор': 918, 'дом': 1025, 'ленинградский': 1929, 'отсутствовать': 2895, 'парковочный': 2963, 'место': 2105, 'житель': 1160, 'постоянно': 3407, 'мучиться': 2224, 'проездом': 3653, 'неоднократно': 2429, 'происходить': 3672, 'дтп': 1075, 'сторона': 4540, 'запретить': 1328, 'оставлять': 2790, 'машина': 2063, 'согласно': 4349, 'установленный': 4988, 'знак': 1420, 'предложение': 3469, 'расширение': 3935, 'авто': 24, 'фото': 5084, 'добрый': 1000, 'день': 945, 'осуществляться': 2806, 'уборка': 4864, 'снег': 4304, 'центр': 5158, 'город': 829, 'гость': 843, 'воспользоваться': 568, 'платный': 3106, 'парковка': 2962, 'движение': 914, 'крайне': 1796, 'затруднить': 1374, 

In [4]:
#определяем метрики для скоринга
scoring = {'Accuracy': make_scorer(accuracy_score), 
           'Precision': make_scorer(precision_score, average = 'weighted'), 
           'Recall': make_scorer(recall_score, average = 'weighted'),
           'f1': make_scorer(f1_score, average = 'weighted'),
           'mcc': make_scorer(matthews_corrcoef)}

In [5]:
#определение наилучших параметров регуляризации (логистическая регрессия)
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Наилучшее значение перекрестной проверки: 0.81
Наилучшие параметры:  {'C': 1}


In [6]:
#кросс-валидация для "мешка слов" (логистическая регрессия)
scores = cross_validate(LogisticRegression(max_iter=10000, C = 1), X_train, y_train, scoring = scoring, cv=5)
print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")
print(f"Коэффициент корреляции Мэтьюза (MCC) : {np.mean(scores['test_mcc'])}")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy перекр проверки: 0.8103686811064369
Precision перекр проверки: 0.8121323869787028
Recall перекр проверки: 0.8103686811064369
f1 перекр проверки: 0.8049008345882317
Коэффициент корреляции Мэтьюза (MCC) : 0.7291115036272011


In [7]:
#определяем модель "tf-idf"
data = data.dropna()
text_train = data['description_stem'].to_list()

vect = TfidfVectorizer(min_df=5).fit(text_train) #игнорировать слова, которые встречаются менее чем в min_df документах
print(vect.vocabulary_)

X_train = vect.transform(text_train)
y_train = data['categories'].to_list()

{'трасса': 4781, 'нижнекамск': 2488, 'чистополь': 5210, 'лесополоса': 1940, 'пгт': 2974, 'камский': 1587, 'поляна': 3317, 'проезжий': 3653, 'частью': 5181, 'опасный': 2727, 'нависать': 2249, 'накрениться': 2291, 'дерево': 950, 'снегопад': 4303, 'мочь': 2191, 'выдержать': 646, 'рухнуть': 4029, 'проходить': 3737, 'автотранспорт': 48, 'двор': 918, 'дом': 1024, 'ленинградский': 1926, 'отсутствовать': 2893, 'парковочный': 2961, 'место': 2100, 'житель': 1157, 'постоянно': 3403, 'мучиться': 2219, 'проездом': 3650, 'неоднократно': 2427, 'происходить': 3669, 'дтп': 1074, 'сторона': 4537, 'запретить': 1325, 'оставлять': 2788, 'машина': 2061, 'согласно': 4347, 'установленный': 4986, 'знак': 1418, 'предложение': 3466, 'расширение': 3933, 'авто': 24, 'фото': 5082, 'добрый': 999, 'день': 945, 'осуществляться': 2804, 'уборка': 4862, 'снег': 4302, 'центр': 5156, 'город': 829, 'гость': 843, 'воспользоваться': 567, 'платный': 3102, 'парковка': 2960, 'движение': 914, 'крайне': 1794, 'затруднить': 1371, '

In [10]:
#определение наилучших параметров регуляризации (логистическая регрессия)
param_grid = {'C': [0.1, 1, 5, 10, 20]}
grid = GridSearchCV(LogisticRegression(max_iter=10000), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Наилучшее значение перекрестной проверки: 0.81
Наилучшие параметры:  {'C': 5}


In [11]:
#кросс-валидация для "tf-idf" (логистическая регрессия)
scores = cross_validate(LogisticRegression(max_iter=10000, C = 5), X_train, y_train, scoring = scoring, cv=5)
print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")
print(f"Коэффициент корреляции Мэтьюза (MCC) : {np.mean(scores['test_mcc'])}")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy перекр проверки: 0.8100751604104375
Precision перекр проверки: 0.8113645420646651
Recall перекр проверки: 0.8100751604104375
f1 перекр проверки: 0.8027188987247428
Коэффициент корреляции Мэтьюза (MCC) : 0.7274548116246816


In [21]:
#Деревья решений
for criterion in ['gini', 'entropy']:
    scores = cross_validate(tree.DecisionTreeClassifier(random_state=0, criterion = criterion, max_depth = 50), X_train, y_train, scoring = scoring, cv=5)
    print(criterion)
    print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
    print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
    print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
    print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")
    print(f"Коэффициент корреляции Мэтьюза (MCC) : {np.mean(scores['test_mcc'])}")

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

gini
Accuracy перекр проверки: 0.7282816532627152
Precision перекр проверки: 0.7280941905885361
Recall перекр проверки: 0.7282816532627152
f1 перекр проверки: 0.7162985521678057
Коэффициент корреляции Мэтьюза (MCC) : 0.6025155294855875


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

entropy
Accuracy перекр проверки: 0.7069848009297856
Precision перекр проверки: 0.7107026801690812
Recall перекр проверки: 0.7069848009297856
f1 перекр проверки: 0.6956600128310214
Коэффициент корреляции Мэтьюза (MCC) : 0.5697711863449805


In [22]:
#Ансамбли деревьев решений
for criterion in ['gini', 'entropy']:
    scores = cross_validate(ensemble.RandomForestClassifier(random_state=0, criterion = criterion, max_depth = 50, n_estimators = 100), X_train, y_train, scoring = scoring, cv=5)
    print(criterion)
    print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
    print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
    print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
    print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")
    print(f"Коэффициент корреляции Мэтьюза (MCC) : {np.mean(scores['test_mcc'])}")

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

gini
Accuracy перекр проверки: 0.7587010968582362
Precision перекр проверки: 0.7587752284256627
Recall перекр проверки: 0.7587010968582362
f1 перекр проверки: 0.7383202427727953
Коэффициент корреляции Мэтьюза (MCC) : 0.6465532591236056


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

entropy
Accuracy перекр проверки: 0.7545127385851056
Precision перекр проверки: 0.7559056212259417
Recall перекр проверки: 0.7545127385851056
f1 перекр проверки: 0.7348438664768369
Коэффициент корреляции Мэтьюза (MCC) : 0.639987639219806


In [ ]:
#Ансамбли деревьев решений
#GradientBoostingClassifier
for criterion in ['mse', 'mae', 'friedman_mse']:
    scores = cross_validate(ensemble.GradientBoostingClassifier(random_state=0, learning_rate = 0.1, criterion = criterion, max_depth = 50, n_estimators = 100), X_train, y_train, scoring = scoring, cv=5)
    print(criterion)
    print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
    print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
    print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
    print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")
    print(f"Коэффициент корреляции Мэтьюза (MCC) : {np.mean(scores['test_mcc'])}")


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py

mse
Accuracy перекр проверки: 0.7780884103475249
Precision перекр проверки: 0.7795735982951835
Recall перекр проверки: 0.7780884103475249
f1 перекр проверки: 0.7747431005609967
Коэффициент корреляции Мэтьюза (MCC) : 0.6835832935250343


SVM

In [23]:
data = data.dropna()
text_train = data['description_stem'].to_list()
data = data.dropna()
text_train = data['description_stem'].to_list()

vect = TfidfVectorizer(min_df=5).fit(text_train) #игнорировать слова, которые встречаются менее чем в min_df документах

X_train = vect.transform(text_train)
y_train = data['categories'].to_list()
print(X_train)

  (0, 5210)	0.23759875524558324
  (0, 5181)	0.23680411438183135
  (0, 4781)	0.21259294725784078
  (0, 4303)	0.21569716593703023
  (0, 4029)	0.2737683091772253
  (0, 3737)	0.1747477488288066
  (0, 3653)	0.13823482585310104
  (0, 3317)	0.2713834168217644
  (0, 2974)	0.2713834168217644
  (0, 2727)	0.14521468879011834
  (0, 2488)	0.21823911726627515
  (0, 2291)	0.2670862003654338
  (0, 2249)	0.2670862003654338
  (0, 2191)	0.12968170106543955
  (0, 1940)	0.28560217794393855
  (0, 1587)	0.2632957868218381
  (0, 950)	0.13641578510228722
  (0, 646)	0.28560217794393855
  (0, 48)	0.21114635226725462
  (1, 5082)	0.1502497894767126
  (1, 4986)	0.16138555030092908
  (1, 4537)	0.11588516045786393
  (1, 4347)	0.18489910783481095
  (1, 3933)	0.23460799602930102
  (1, 3669)	0.18824855120527875
  :	:
  (23669, 942)	0.017361028453053453
  (23669, 918)	0.018748587082298044
  (23669, 914)	0.036808326338292945
  (23669, 906)	0.02286963598440411
  (23669, 904)	0.012095086462754555
  (23669, 840)	0.0156877282

In [5]:
#определение наилучших параметров регуляризации (SVM)
param_grid = {'C': [0.1, 1, 5, 10, 20]}
grid = GridSearchCV(SVC(probability=False, kernel='rbf'), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Наилучшее значение перекрестной проверки: {:.2f}".format(grid.best_score_))
print("Наилучшие параметры: ", grid.best_params_)


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default 

Наилучшее значение перекрестной проверки: 0.74
Наилучшие параметры:  {'C': 20}


In [6]:
#кросс-валидация для "tf-idf" (SVM)
scores = cross_validate(SVC(probability=False, kernel='rbf', C = 20), X_train, y_train, scoring = scoring, cv=5)
print(f"Accuracy перекр проверки: {np.mean(scores['test_Accuracy'])}")
print(f"Precision перекр проверки: {np.mean(scores['test_Precision'])}")
print(f"Recall перекр проверки: {np.mean(scores['test_Recall'])}")
print(f"f1 перекр проверки: {np.mean(scores['test_f1'])}")
print(f"Коэффициент корреляции Мэтьюза (MCC) : {np.mean(scores['test_mcc'])}")

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or '

Accuracy перекр проверки: 0.7397281551423884
Precision перекр проверки: 0.7324895568760004
Recall перекр проверки: 0.7397281551423884
f1 перекр проверки: 0.7155344870194187
Коэффициент корреляции Мэтьюза (MCC) : 0.6167290967734356
